### Importing the needed liberaries

In [ ]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# the dataset
df_train=pd.read_csv('../input/titanic/train.csv')


### (EDA) Eploratory Data Analysis

In [ ]:
df_train.head()

In [ ]:
df_train.info()

 there are missing values in the columns Age and cabin columns
 
 
 for more investigations :


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
sns.heatmap(df_train.isnull(),yticklabels=False,cbar=False)

plt.title('the missing values distribution in the data',fontsize=16)
plt.show()

the data in the column cabin is very big so we will drop it
in the other hand the data in the column Age is small and will distributed in the data so we will try if we can fill it

### Data Cleaning

In [ ]:
# droping the column cabin
df_witout_cabin=df_train.drop('Cabin',axis=1)

In [ ]:
#cheking for the rest of the columns
df_witout_cabin.columns

we will expolre the relation between Age column 
and the rest of the data inorder to fill it or deciding to drop it 

In [ ]:
# we will count the percent of the missing values in order to know if the droping option is avliable for us
df_witout_cabin['Age'].isnull().sum()

In [ ]:
len(df_witout_cabin['Age'])

In [ ]:
percent_of_nulls=177*100/891

percent_of_nulls

the percent is greater than 5% so if we drop the null values in the Age column, then we will not be able to use it in our model unless we find a way to fillin the null values 

now we will explore the data;

In [ ]:
sns.boxplot(data=df_witout_cabin,y='Age',x='Sex')

thre is no relation between Age and SEX

In [ ]:
sns.jointplot(data=df_witout_cabin,x='Age',y='Fare',kind='hex')

In [ ]:
sns.boxplot(data=df_witout_cabin,y='Age',x='Pclass')

the figure shows that people with age around 38 tends to buy expensive ticket in the first class and people with age  around 29 tend to buy tickets in the second class however people with age around 24 tend to buy cheater tickets

**so**, we can fillin null values in the Age column by informations from pclass ****

In [ ]:
def f(x):
    Age=x[0]
    Pclass=x[1]
    if pd.isnull(Age):
        if Pclass==1:
            return 38
        elif Pclass==2:
            return 29
        else:
            return 24
    else:
        return Age

In [ ]:
df_witout_cabin['Age']=df_witout_cabin[['Age','Pclass']].apply(f,axis=1)

In [ ]:
sns.heatmap(df_witout_cabin.isnull(),yticklabels=False,cbar=False)

plt.title('the missing values distribution in the data',fontsize=16)
plt.show()

In [ ]:
df_witout_cabin.isnull().sum()

In [ ]:
df_clean=df_witout_cabin.dropna()

In [ ]:
df_clean.isnull().sum()

In [ ]:
sns.displot(data=df_clean,x='Fare')

In [ ]:
df_cleaned=df_clean[(df_clean['Fare']<200)]

In [ ]:
sns.displot(data=df_cleaned,x='Fare')

In [ ]:
df_cleaned.columns

In [ ]:
sns.countplot(data=df_cleaned,x='Survived',hue='Sex')

In [ ]:
sns.displot(data=df_cleaned,x='Survived')

In [ ]:
sns.pairplot(df_cleaned,corner=True)

In [ ]:
df_train.describe(include='all')

### Data Preprocessing

In [ ]:
df_needed=df_cleaned[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp',
       'Parch', 'Fare', 'Embarked']]
df_needed.head()

In [ ]:
df_dummies=pd.get_dummies(df_needed,drop_first=True)

In [ ]:
df_dummies.head()

In [ ]:
X_train=df_dummies.drop('Survived',axis=1)
y_train=df_dummies['Survived']

In [ ]:
X_train.head()

In [ ]:
y_train.head()

### Choosing The correct Featues For Our Model

In [ ]:
import statsmodels.api as sm


from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)

In [ ]:
x=sm.add_constant(X_train)
model=sm.Logit(y_train,x)
result=model.fit()

result.summary()

the feature with p_value greater than 0.005 has no statsticaly scnificance and must be deleted according to the back_ward_elimination_rule  

but we will jut one feature at the step

In [ ]:
X_=X_train.drop(X_train[['Embarked_Q','Embarked_S']],axis=1)


In [ ]:
X_.head()

In [ ]:
x1=sm.add_constant(X_)
model=sm.Logit(y_train,x1)
result=model.fit()

result.summary()

In [ ]:
X_trial=X_.drop('Parch',axis=1)


In [ ]:
x2=sm.add_constant(X_trial)
model=sm.Logit(y_train,x2)
result=model.fit()

result.summary()

In [ ]:
X_final=X_trial.drop('Fare',axis=1)


In [ ]:
x3=sm.add_constant(X_final)
model=sm.Logit(y_train,x3)
result=model.fit()

result.summary()

### Building The model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logmodel=LogisticRegression()

In [ ]:
logmodel.fit(X_final,y_train)

### Preparing The test data before using it

In [ ]:
df_test = pd.read_csv('../input/titanic/test.csv')
df_test.head()

In [ ]:
df_test.describe()

In [ ]:
df_test['Age']=df_test[['Age','Pclass']].apply(f,axis=1)

In [ ]:
df_test_needed=df_test[['Pclass', 'Sex', 'Age', 'SibSp']]
df_test_needed.head()

In [ ]:
df_test_needed.describe()

In [ ]:
x_test_dummies=pd.get_dummies(df_test_needed,drop_first=True)
x_test_dummies.head()

### Getting the predictions

In [ ]:
predictions=logmodel.predict(x_test_dummies)

In [ ]:
pred=pd.DataFrame(predictions,columns=['pred'])
pred.head()

### Evaluating the predictions

In [ ]:
sub = pd.read_csv('../input/titanic/gender_submission.csv')
sub_compare=sub.copy()
sub_compare['predictions']=predictions
#pd.set_option('display.max_rows', None)
sub_compare.tail(20)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
sub_compare.columns

In [ ]:
confusion_matrix(sub_compare['Survived'],pred)

In [ ]:
final_result=pd.DataFrame(confusion_matrix(sub_compare['Survived'],pred),index=[['actual No','actual Yes']],columns=[['predicted No','predicted Yes']])
final_result

In [ ]:
classification_report(sub_compare['Survived'],pred)

### Final submition

In [ ]:
Final_submition=sub_compare.drop('Survived',axis=1)
Final_submition.to_csv('my_submition',index=False)

### If you liked the notebook, please upvote for it!